In this example we are going to scrape [Iowa's Professional Licensing](https://ia-plb.my.site.com/LicenseSearchPage) website for appraisal management companies. (Why? Who knows, why *not?*)

Traditionally Python programmers use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape content from the interent. Instead of being *traditional*, we're going to use [Playwright](https://playwright.dev/python/), a **browser automation tool**! This means you actually control the browser! Filling out forms, clicking buttons, downloading documents... it's magic!!!✨✨✨

# Iowa Appraisal Management Companies

- Dropdowns
- Clicking 'Next page' button again and again
- Combining dataframes

## Installation

We need to install a few tools first! Remove the `#` and run the cell to install the Python packages and browsers that we'll need for our scraping adventure.

In [ ]:
# %pip install lxml html5lib beautifulsoup4 pandas
# %pip install --quiet playwright
# !playwright install

## Opening up the browser and visiting our destination


In [ ]:
from playwright.async_api import async_playwright

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

# Create a new browser window
page = await browser.new_page()

In [ ]:
await page.goto("https://ia-plb.my.site.com/LicenseSearchPage")

## Selecting an option from a dropdown

You always start with `await page.locator("select").select_option("whatever option you want")`. You'll probably get an error because there are multiple dropdowns on the page, but Playwright doesn't know which one you want to use! Just read the error and figure out the right one.

In [ ]:
# await page.locator("select").select_option("Appraisal Management Company")
await page.get_by_label("Licensing Board:").select_option("Appraisal Management Company")

In [ ]:
# await page.get_by_text("Search").click()
await page.locator("input[name=\"j_id0\\:j_id1\\:j_id14\\:j_id73\"]").click()

## Grab the tables from the page

[Pandas](https://pandas.pydata.org/) is the Python equivalent to Excel, and it's great at dealing with tabular data! Often the data on a web page that looks like a spreadsheet can be read with `pd.read_html`.

You use `await page.content()` to save the contents of the page, then feed it to `read_html` to find the tables. `len(tables)` checks the number of tables you have, then you manually poke around to see which one is the one you're interested in. `tables[0]` is the first one, `tables[1]` is the second one, and so on...

In [ ]:
import pandas as pd
from io import StringIO

html = await page.content()
tables = pd.read_html(StringIO(html))
len(tables)

In [ ]:
tables[0]

## Clicking "next page" one

Just like using a dropdown, select box or button, we'll use `page.get_by_text` to try to select the button.

We add `timeout=5000` to wait 5 seconds before confirming it isn't there.

In [ ]:
# page.get_by_text("Next Page").click()
await page.locator("a:has-text('Next Page')").click(timeout=5000)

## Clicking "next page" until it disappears

Eventually the "next page" link disappears, and Python starts *screaming*. We use `try/except` down below to say "don't worry little baby, it's okay, we'll just finish up if the button is gone."

In [ ]:
# all of our data will end up going here
all_data = pd.DataFrame()

while True:
    # Get all of the tables on the page
    html = await page.content()
    tables = pd.read_html(StringIO(html))

    # Get the table (and edit if necessary)
    df = tables[0]

    # Add the tables on this page to the big list of stuff
    all_data = pd.concat([all_data, df], ignore_index = True)
    try:
        await page.locator("a:has-text('Next Page')").click(timeout=5000)
    except:
        break

In [ ]:
all_data

## Saving the results

Now we'll save it to a CSV file! Easy peasy.

In [ ]:
all_data.to_csv("output.csv", index=False)